In [1]:
from dictionary_learning.trainers.top_k import AutoEncoderTopK
import torch
from utils import MODELS_DIR, INPUT_DATA_DIR
from transformers import set_seed
from transformers import MusicgenForConditionalGeneration, AutoProcessor
from IPython.display import Audio, display

device = "cuda:1"
model_name = "facebook/musicgen-medium"
max_tokens = 200
base_dir = INPUT_DATA_DIR / "music-bench" / "datashare-instruments"
model_sr = 32000

In [ ]:
model = MusicgenForConditionalGeneration.from_pretrained(model_name).to(device)
processor = AutoProcessor.from_pretrained(model_name)
ae = AutoEncoderTopK.from_pretrained(
    MODELS_DIR / "medium-sae-trivial-medium-sae-ee3b" / "16" / "trainer_0" / "checkpoints" / "ae_71100.pt"
).to(device)

KeyboardInterrupt: 

In [ ]:
model.eval()

MusicgenForConditionalGeneration(
  (text_encoder): T5EncoderModel(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): L

In [ ]:
val = -1

with torch.no_grad():
    raggae_out = torch.load(INPUT_DATA_DIR / "unlearn" / "raggae.pt")
    raggae_mu = torch.load(INPUT_DATA_DIR / "unlearn" / "raggae_mu.pt")
    q = 0.99
    thr = torch.quantile(raggae_out.float(), q).cpu()
    x = raggae_out.detach().cpu()
    x[x > thr] = -100
    ind = x.topk(3).indices
    mask = torch.zeros(x.shape[0])
    mask[ind] = 1
    intervention_mask = raggae_mu.detach().cpu()
    intervention_mask[~mask.bool()] = 1 / val
    intervention_mask *= val
intervention_mask = intervention_mask.to(device)

In [ ]:
ind

tensor([2357,  345, 1660])

In [ ]:
hook_point = model.decoder.get_submodule("model.decoder.layers.16")
y = [None]
i = []
set_seed(42)


def hook(module, input, output):
    # set_seed(42)
    i.append(input)
    z = ae.encode(output[0])
    # z[:, :, ind] *= -0.1
    z = intervention_mask * z
    return (ae.decode(z), output[1])


remove_handle = hook_point.register_forward_hook(hook)
inputs = processor(
    text=[
        "A laid-back reggae groove with warm bassline, off-beat guitar skanks, smooth organ chords, and relaxed drum rhythm, tropical and sunny atmosphere."
    ]
    * 3,
    padding=True,
    return_tensors="pt",
)
# dataset = load_dataset("sanchit-gandhi/gtzan", split="train", streaming=True)
# sample = next(iter(dataset))["audio"]

# # take the first half of the audio sample
# sample["array"] = sample["array"][: len(sample["array"]) // 2]

# inputs = processor(
#     audio=sample["array"],
#     sampling_rate=sample["sampling_rate"],
#     text=["80s blues track with groovy saxophone"],
#     padding=True,
#     return_tensors="pt",
# )
inputs = {k: v.to(device) for k, v in inputs.items()}
audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)
for x in audio_values.detach().cpu():
    display(Audio(x, rate=model.config.sampling_rate))
remove_handle.remove()
audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)
for x in audio_values.detach().cpu():
    display(Audio(x, rate=model.config.sampling_rate))

In [ ]:
i[0]

(tensor([[[-15.1285, -15.1449, -15.1150,  ..., -15.1078, -15.1342, -15.1332]],
 
         [[-15.1285, -15.1449, -15.1150,  ..., -15.1078, -15.1342, -15.1332]],
 
         [[-15.1285, -15.1449, -15.1150,  ..., -15.1078, -15.1342, -15.1332]],
 
         [[-15.1273, -15.1086, -15.1324,  ..., -15.0857, -15.1260, -15.0986]],
 
         [[-15.1273, -15.1086, -15.1324,  ..., -15.0857, -15.1260, -15.0986]],
 
         [[-15.1273, -15.1086, -15.1324,  ..., -15.0857, -15.1260, -15.0986]]],
        device='cuda:1'),)

In [ ]:
len(i)

256